In [1]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
import pandas as pd

cred = credentials.Certificate('key.json')
firebase_admin.initialize_app(cred)
db = firestore.client()

In [1]:
def addFarm(data):
    # before adding, check if another value with the same Name & Adress exists. Right now we just add another
    dbRef = db.collection(u'farms').document()
    dbRef.set(data)

def addFarmsFromDF(df):
    for i in range(0, len(df)):
        data = {
        u'name': df.Name[i],
        u'adress': df.Adress[i],
        u'website': df.Website[i]
        }
        
        addFarm(data)
        
def queryFarms(queryValue, queryField = u"Name", operator = u"=="):
    docs = db.collection(u'farms').where(queryField, operator, queryValue).stream()

    for doc in docs:
        print(u'{} => {}'.format(doc.id, doc.to_dict()))
        
def addGroup(data):
    # before adding, check if another value with the same Name & Adress exists. Right now we just add another
    dbRef = db.collection(u'groups').document()
    dbRef.set(data)
        
def addGroupsFromDF(df):
    for i in range(0, len(df)):
        data = {
        u'id': df.index[i],
        u'location': df.location[i],
        u'farmsIncluded': df.farmsIncluded[i],
        u'numberInGroup': df.numberInGroup[i]
        }
        
        addGroup(data)

In [2]:
data = {
        u'name': u'Tims Apple Place',
        u'state': u'First Road, 2004 Perth',
        u'country': u'timsfarm.co.au'
    }

In [ ]:
df = pd.read_excel("Data/Data_27-04-2020.xlsx")

In [6]:
addFarm(data)

In [11]:
queryFarms(u"Tims Apple Place", u"Name", u"==")